In [56]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torchvision
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision.transforms import ToTensor
import math
from tqdm import tqdm

In [6]:
print(torch.__version__, torchvision.__version__)
print("cuda available:", torch.cuda.is_available())
print("cudnn version:", torch.backends.cudnn.version())
# Get device name
if torch.cuda.is_available():
    print("Device name:", torch.cuda.get_device_name(0))

2.8.0+cu128 0.23.0+cu128
cuda available: True
cudnn version: 91002
Device name: NVIDIA GeForce RTX 2060


In [ ]:
torch.set_default_dtype(torch.float64)
#device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"

'cuda'

In [49]:
class Net(torch.nn.Module):
    def __init__(self, d_in = 4, d_hidden = 64):
        super().__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(d_in, d_hidden), torch.nn.SiLU(),
            torch.nn.Linear(d_hidden, d_hidden), torch.nn.SiLU(),
            torch.nn.Linear(d_hidden, 1)
        )
    def forward(self, x):  # x: [B,4] -> [B,1]
        return self.mlp(x)

net = Net().to(device)
opt = torch.optim.AdamW(net.parameters(), lr=1e-3)

In [50]:
K = torch.tensor(100.0, device=device)

def one_path_label_and_grads(x, antithetic=True):
    """
    x: [B,4] with grads enabled on inputs.
    Returns: y (price) [B,1], g (dy/dx) [B,4]
    """
    B = x.shape[0]
    S0, vol, r, T = [x[:,i:i+1] for i in range(4)]
    # One Normal per sample (+ antithetic for variance reduction)
    Z = torch.randn(B, 1, device=device)
    if antithetic:
        Z = torch.cat([Z, -Z], dim=1)  # [B,2]
    # Expand params to both paths
    Texp = T
    drift = (r - 0.5*vol**2) * Texp
    diff  = vol * torch.sqrt(Texp + 1e-12)
    ST = S0 * torch.exp(drift + diff * Z)           # [B,2]
    payoff = torch.clamp(ST - K, min=0.0)           # [B,2]
    disc = torch.exp(-r * Texp)                     # [B,1]
    y = disc * payoff.mean(dim=1, keepdim=True)     # [B,1]
    # Pathwise derivatives via autograd
    grads = torch.autograd.grad(
        y.sum(), (S0, vol, r, T), create_graph=False, retain_graph=False
    )
    g = torch.cat([gi for gi in grads], dim=1)      # [B,4]
    return y.detach(), g.detach()

# --- Training loop: online sampling, single path per sample ---
def sample_batch(B):
    # Choose your training distribution for states (very important!)
    S0 = torch.exp(torch.randn(B)*0.25 + math.log(100.0))
    vol = torch.clamp(torch.randn(B)*0.05 + 0.2, 0.05, 0.6)
    r   = torch.clamp(torch.randn(B)*0.01 + 0.01, -0.01, 0.05)
    T   = torch.clamp(torch.rand(B) * 2.0, 0.05, 2.0)
    x = torch.stack([S0, vol, r, T], dim=1).to(device).requires_grad_(True)
    return x

def featurize(x, K):
    # x: [B,4] = (S0, vol, r, T)
    S0, vol, r, T = [x[:,i:i+1] for i in range(4)]
    m   = torch.log(S0 / K)         # log-moneyness
    v   = torch.log(vol)            # log-vol
    tau = torch.sqrt(T + 1e-12)     # time feature
    z = torch.cat([m, v, r, tau], dim=1)
    return z

In [62]:
import torch
from tqdm import tqdm

torch.set_default_dtype(torch.float64)
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- tiny helper: per-batch scale for gradient loss (robust to outliers) ---
def robust_scale(g, eps=1e-8):
    # median absolute deviation-like scale per coordinate
    med = g.median(dim=0).values
    mad = (g - med).abs().median(dim=0).values + eps
    # Prevent too-small scales
    return torch.clamp(mad, min=1e-3)

# --- model ---
class Net(torch.nn.Module):
    def __init__(self, d_in=4, d_hidden=32):
        super().__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(d_in, d_hidden), torch.nn.SiLU(),
            torch.nn.Linear(d_hidden, d_hidden), torch.nn.SiLU(),
            torch.nn.Linear(d_hidden, 1)
        )
    def forward(self, x):
        return self.mlp(x)

net = Net().to(device)
opt = torch.optim.AdamW(net.parameters(), lr=3e-4, weight_decay=0.0)

# --- training ---
steps = 5000
B = 3000
alpha = 1.0

beta_start, beta_end = 5.0, 1.0      # start gradient-heavy, then ease
warmup_steps = int(0.6 * steps)      # linear schedule for beta

pbar = tqdm(range(steps), desc="Training", ncols=100)
for step in pbar:
    # 1) sample a batch of raw states
    x = sample_batch(B)                              # [B,4]; requires_grad set inside sample_batch
    y, g = one_path_label_and_grads(x)               # labels wrt RAW x (both detached inside function)

    # 2) forward pass on a fresh, grad-enabled copy of x
    x_ = x.detach().clone().requires_grad_(True)
    yhat = net(x_)

    # 3) model gradient wrt x (to match simulator pathwise grads)
    grads_pred = torch.autograd.grad(yhat.sum(), x_, create_graph=True)[0]

    # 4) simple, robust scaling for gradient loss (per coordinate)
    s = robust_scale(g).unsqueeze(0)                 # [1,4]
    grad_res = (grads_pred - g) / s
    grad_loss = (grad_res**2).mean()

    # 5) price loss (squared error)
    val_loss = ((yhat - y)**2).mean()

    # 6) beta schedule
    if step < warmup_steps:
        t = step / max(1, warmup_steps)
        beta = beta_start * (1 - t) + beta_end * t
    else:
        beta = beta_end

    loss = alpha*val_loss + beta*grad_loss

    # 7) step
    opt.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
    opt.step()

    # 8) pretty progress
    if (step % 50) == 0:
        pbar.set_postfix(
            loss=f"{loss.item():.2f}",
            val=f"{val_loss.item():.2f}",
            grad=f"{grad_loss.item():.2f}",
            beta=f"{beta:.2f}"
        )


Training: 100%|██| 5000/5000 [00:35<00:00, 139.43it/s, beta=1.00, grad=10.99, loss=55.44, val=44.45]


In [ ]:
# class ResidualBlock(torch.nn.Module):
#     def __init__(self, d, width):
#         super().__init__()
#         self.lin1 = torch.nn.Linear(d, width)
#         self.lin2 = torch.nn.Linear(width, d)
#         self.act  = torch.nn.SiLU()
#     def forward(self, x):
#         h = self.act(self.lin1(x))
#         h = self.lin2(h)
#         return x + h

# class Net(torch.nn.Module):
#     def __init__(self, d_in=4, width=64, nblocks=3):
#         super().__init__()
#         self.inp = torch.nn.Linear(d_in, width)
#         self.blocks = torch.nn.ModuleList([ResidualBlock(width, width) for _ in range(nblocks)])
#         self.out = torch.nn.Linear(width, 1)
#         self.act = torch.nn.SiLU()
#     def forward(self, z):          # z are features from featurize(...)
#         h = self.act(self.inp(z))
#         for b in self.blocks:
#             h = self.act(b(h))
#         return self.out(h)

# net = Net().to(device)
# opt = torch.optim.AdamW(net.parameters(), lr=3e-4, weight_decay=0.0)


In [ ]:
# K = torch.tensor(100.0, device=device)

# alpha, beta = 1.0, 5.0
# for step in range(5000):
#     B = 512
#     x = sample_batch(B)                     # your sampler; requires_grad already set inside
#     y, g_x = one_path_label_and_grads(x)    # labels + pathwise grads wrt RAW x

#     # --- features ---
#     z = featurize(x, K)                     # [B,4]  (no detach; we only need grads wrt z for the model)

#     # --- forward ---
#     z_ = z.detach().clone().requires_grad_(True)
#     yhat = net(z_)

#     # --- model Jacobian wrt features z ---
#     grads_pred_z = torch.autograd.grad(yhat.sum(), z_, create_graph=True)[0]

#     # --- loss: value + gradient match ---
#     # NOTE: g_x are grads wrt RAW x. Because the model is f(z(x)), matching grads wrt z is fine:
#     #       we’re asking the net to learn d/dz of f, which is coherent as long as we always feed z.
#     #       (You could also transform g_x to z-space via chain rule, but here we keep it simple:
#     #        train the model as a function of z and match its ∂/∂z to simulator ∂y/∂z computed via autograd through featurize.)
#     # Compute simulator grads wrt z using autograd through featurize:
#     #   z = φ(x), y = y(x)        -> ∂y/∂z = (∂y/∂x) * (∂x/∂z)
#     # It’s a bit cleaner to recompute ∂y/∂z directly:
#     x_ = x.detach().clone().requires_grad_(True)
#     z_for_label = featurize(x_, K)
#     # y doesn't depend on z_for_label directly, but on x_; we need ∂y/∂z = (∂y/∂x) * (∂x/∂z)
#     # Instead of manual chain-rule, get ∂y/∂z by differentiating y wrt z_for_label via implicit graph:
#     y_for_grad, _ = one_path_label_and_grads(x_)      # y(x_)
#     gy_wrt_z = torch.autograd.grad(y_for_grad.sum(), z_for_label, retain_graph=False, create_graph=False)[0].detach()

#     val_loss  = ((yhat - y)**2).mean()
#     grad_loss = ((grads_pred_z - gy_wrt_z)**2).mean()
#     loss = alpha*val_loss + beta*grad_loss

#     opt.zero_grad()
#     loss.backward()
#     torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
#     opt.step()

#     if step % 500 == 0:
#         print(step, f"loss={float(loss):.4f}  val={float(val_loss):.4f}  grad={float(grad_loss):.4f}")


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [55]:
alpha, beta = 1.0, 1.0         # balance value vs gradient fitting
for step in range(10000):
    B = 100
    x = sample_batch(B)
    y, g = one_path_label_and_grads(x, False)
    # Forward + model gradients
    x_ = x.detach().clone().requires_grad_(True)
    yhat = net(x_)
    # ∇_x f(x) via autograd
    grads_pred = torch.autograd.grad(
        yhat.sum(), x_, create_graph=False, retain_graph=True
    )[0]
    # Sobolev loss
    loss = alpha*((yhat - y)**2).mean() + beta*((grads_pred - g)**2).mean()
    opt.zero_grad(); loss.backward(); opt.step()
    if step % 200 == 0:
        print(step, float(loss))

0 4259.559045539481
200 4177.255755677336
400 3327.7115667896674
600 2611.4894760969546
800 2624.1089982337803
1000 3184.893391791928
1200 2600.5811784652283
1400 3264.611954987147
1600 4982.693417683873
1800 2846.061057497064
2000 4042.258212188225
2200 2897.1457562536843
2400 3262.3354711007596
2600 4932.153302748594
2800 3387.4655739048667
3000 3726.4271660410222
3200 5310.121768827692
3400 3659.1867767549656
3600 4583.559063653327
3800 2298.277520647318
4000 3670.0058195820384
4200 1745.3328547701385
4400 4429.387784103051
4600 3093.0967924813913
4800 3319.23684096691
5000 2553.6353547932945
5200 5756.225714453664
5400 4182.995632913563
5600 3868.662184963603
5800 3020.6810771064115
6000 2011.8831412864652
6200 2931.321377236097
6400 3589.226707404174
6600 2547.6018627409503
6800 1924.7501741159394
7000 3795.262267549007
7200 2998.3106842399984
7400 2026.5782633762733
7600 3143.482910427123
7800 3991.9112592545052
8000 3677.3876425949743
8200 4763.023528997892
8400 2142.89107640208

In [30]:
import torch, math

# --------- running scalers ---------
# Standardize inputs using running mean/var (EMA)
class RunningStandardizer:
    def __init__(self, d, eps=1e-8, decay=0.01, device="cpu"):
        self.mu = torch.zeros(d, device=device)
        self.nu = torch.ones(d, device=device)      # second moment
        self.decay = decay
        self.eps = eps
        self.t = 0

    def update(self, x):  # x: [B,d]
        with torch.no_grad():
            self.t += 1
            m = x.mean(dim=0)
            v = (x**2).mean(dim=0)
            self.mu = (1 - self.decay)*self.mu + self.decay*m
            self.nu = (1 - self.decay)*self.nu + self.decay*v

    @property
    def std(self):
        return torch.sqrt(torch.clamp(self.nu - self.mu**2, min=self.eps))

    def transform(self, x):
        return (x - self.mu) / self.std

    def inv_transform(self, z):
        return z * self.std + self.mu

# Diagonal whitening for gradient residuals: weight by 1/var
class GradWhitening:
    def __init__(self, d, eps=1e-8, decay=0.01, device="cpu"):
        self.m2 = torch.ones(d, device=device)  # E[g^2]
        self.decay = decay
        self.eps = eps
    def update(self, g):      # g: [B,d]
        with torch.no_grad():
            self.m2 = (1 - self.decay)*self.m2 + self.decay*(g**2).mean(dim=0)
    @property
    def inv_var(self):
        return 1.0 / torch.clamp(self.m2, min=self.eps)

device = next(net.parameters()).device
x_scaler = RunningStandardizer(d=4, device=device)
gw = GradWhitening(d=4, device=device)

huber_delta = 5.0
def huber(res, delta=huber_delta):
    absr = torch.abs(res)
    quad = torch.minimum(absr, torch.tensor(delta, device=res.device))
    lin  = absr - quad
    return 0.5*quad**2 + delta*lin

alpha, beta0, beta1 = 1.0, 5.0, 0.5  # start gradient-heavy, then anneal down
total_steps = 5000
lr = 3e-4
for g in opt.param_groups: g['lr'] = lr

for step in range(total_steps):
    B = 512
    x_raw = sample_batch(B)                    # [B,4], raw
    x_scaler.update(x_raw)
    xz = x_scaler.transform(x_raw.detach())    # standardized inputs

    # Labels and pathwise grads w.r.t. RAW x
    y, g_raw = one_path_label_and_grads(x_raw) # detached

    # Convert label-gradients to standardized coordinates:
    # z = (x - mu)/std  =>  ∂/∂z = std * ∂/∂x   (elementwise)
    std = x_scaler.std.unsqueeze(0)            # [1,4]
    g_z = g_raw * std                          # chain rule

    # Update whitening on g_z
    gw.update(g_z)

    # Model forward on standardized inputs
    xz_ = xz.detach().clone().requires_grad_(True)
    yhat = net(xz_)

    # Model gradient wrt standardized inputs
    grads_pred = torch.autograd.grad(yhat.sum(), xz_, create_graph=True)[0]

    # Loss pieces
    beta = beta0 * (1 - step/ (0.6*total_steps)) + beta1 * (step/(0.6*total_steps))
    beta = float(max(beta, beta1))

    # Huber on value residual (robust to one-path noise)
    val_res = (yhat - y)
    val_loss = huber(val_res).mean()

    # Whitened gradient loss: sum_j w_j * (Δg_j)^2
    w = gw.inv_var.unsqueeze(0)                 # [1,4]
    grad_res = grads_pred - g_z
    grad_loss = (w * (grad_res**2)).mean()

    loss = alpha*val_loss + beta*grad_loss

    opt.zero_grad()
    torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
    loss.backward()
    opt.step()

    if step % 200 == 0:
        with torch.no_grad():
            # Log per-term to see who dominates
            print(f"{step:4d} | loss {loss.item():.3f}  val {val_loss.item():.3f}  grad {grad_loss.item():.3f}  beta {beta:.2f}")


   0 | loss 2320.607  val 1436.659  grad 176.790  beta 5.00
 200 | loss 28.551  val 26.162  grad 0.508  beta 4.70
 400 | loss 16.766  val 15.631  grad 0.258  beta 4.40
 600 | loss 14.467  val 13.221  grad 0.304  beta 4.10
 800 | loss 13.258  val 11.754  grad 0.396  beta 3.80
1000 | loss 15.332  val 13.015  grad 0.662  beta 3.50
1200 | loss 16.900  val 13.233  grad 1.146  beta 3.20
1400 | loss 19.008  val 14.852  grad 1.433  beta 2.90
1600 | loss 17.131  val 14.438  grad 1.036  beta 2.60
1800 | loss 15.008  val 12.872  grad 0.929  beta 2.30
2000 | loss 16.986  val 14.950  grad 1.018  beta 2.00
2200 | loss 12.878  val 11.421  grad 0.857  beta 1.70
2400 | loss 15.321  val 13.571  grad 1.251  beta 1.40
2600 | loss 13.299  val 12.432  grad 0.789  beta 1.10
2800 | loss 13.820  val 13.037  grad 0.979  beta 0.80
3000 | loss 11.024  val 10.651  grad 0.745  beta 0.50
3200 | loss 14.911  val 14.394  grad 1.035  beta 0.50
3400 | loss 12.989  val 12.472  grad 1.034  beta 0.50
3600 | loss 10.239  va